# Forward pass for Vanilla RNN
Taken from here: https://gist.github.com/karpathy/d4dee566867f8291f086

In [1]:
import numpy as np

### Reading the data

In [5]:
data = open('resources/shakespeare.txt', 'r').read() 

In [6]:
print(data[:610])

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.



### Encoding characters

In [7]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
# char-to-int encoding
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print('data has {} characters, {} unique'.format(data_size, vocab_size))

data has 9837 characters, 57 unique


### RNN basics

#### RNN Structure
<img src="resources/rnn_schema.png" alt="Drawing" width="500"/>

#### Components of RNN
Taken from here: https://cs224d.stanford.edu/lecture_notes/LectureNotes4.pdf

<img src="resources/rnn_components.png" alt="Drawing" width="500"/>

Let's say that we defined the hyperparameters for the RNN layer as follows:<br>

**hidden size** = 100<br>
**sequence length** = 25


So, what is the hidden size? 

In the example below we set the sequence length to 25. When we unroll the RNN layer, there will be 25 hidden states $h_{t}$. <br> Each hidden state will have the specified size of 100. So hidden size is just the size of every $h_{t}$. 

The output $h_{t}$ depends on two inputs:  
(1) previous hidden state $h_{t-1}$  
(2) the current element of the sequence $x_t$

Regular neural networks use only one element $x$ as an input, <br>
so there were only one matrix of parameters $W$. 

But in the case of RNN the output depends on TWO inputs, so we should have TWO parameter matrices instead of just one:

$$h_{t} = tanh(W^{hh}*h_{t-1} + W^{hx}*x_{t} + bias)$$


$W^{hx}$ is associated with the inputs.

$W^{hh}$ is associated with the previous hidden state.<br><br>



$h_{t}$ should have dimensions of $(hidden\_size, 1)$, it is $(100, 1)$ in our case. <br>
In order for this to happen, both terms $W^{hh}*h_{t-1}$ and $W^{hx}*x_{t}$ should have the same dimension of $(100, 1)$. <br>

We already know that $h_{t-1}$ is $(100, 1)$, then $W^{hh}$ should have dimensions of $(100, 100)$, because:
<br>
$$(100, 100) * (100, 1) = (100, 1)$$
<br>
The $W^{hx}$ should have dimensions of $(100, vocab\_size)$, because:
<br>
$$(100, vocab\_size) * (vocab\_size, 1) = (100, 1)$$

### Defining the hyperparameters

In [8]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

### Initializing the weights

In [9]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [10]:
print('Input to hidden weights: {}'.format(Wxh.shape))
print('Hidden to hidden weights: {}'.format(Whh.shape))
print('Hidden to output weights: {}\n'.format(Why.shape))
print('Hidden bias weights: {}'.format(bh.shape))
print('Output bias weights: {}'.format(by.shape))

Input to hidden weights: (100, 57)
Hidden to hidden weights: (100, 100)
Hidden to output weights: (57, 100)

Hidden bias weights: (100, 1)
Output bias weights: (57, 1)


### Extracting one sequence
For the purpose of demonstration, I am using only one sequence of length 25 in the forward pass. 
*inputs* - one sequence of length 25
*targets* - one sequence of length 25 shifted by 1 over the *inputs*

In [11]:
p = 0
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

# word 'from'
for i in range(3):
    print('{}({}) -> {}({})'.format(ix_to_char[inputs[i]], inputs[i], ix_to_char[targets[i]], targets[i]))

F(45) -> r(41)
r(41) -> o(14)
o(14) -> m(24)


### RNN Forward pass

In [12]:
# defining the previous hidden state
# it equals to zeros since for the fisrt element there is no preious hidden state
hprev = np.zeros((hidden_size,1))

In [13]:
# xs - contains one-hot encoded character in a sequence for timestep t
# hs - contains hidden states for timestep t
# ys - contains output for timestep t 
# ps - contains probabilities for timestep t 
xs, hs, ys, ps = {}, {}, {}, {}

In [14]:
hs[-1] = np.copy(hprev)
loss = 0

In [15]:
for t in range(seq_length):
    #print('Timestep: {}'.format(t+1))
    xs[t] = np.zeros((vocab_size,1)) # one-hot
    xs[t][inputs[t]] = 1 # one-hot    
    first_term = np.dot(Wxh, xs[t])
    second_term = np.dot(Whh, hs[t-1])
    hs[t] = np.tanh(first_term + second_term + bh)
    ys[t] = np.dot(Why, hs[t]) + by
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0])